In [1]:
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist']


INFO 2025-02-03 23:15:54,434 storage 使用 COSStorage 初始化存储
INFO 2025-02-03 23:15:54,435 storage 初始化 COSStorage
INFO 2025-02-03 23:15:54,435 storage Storage backend initialization with args: (), kwargs: {}
INFO 2025-02-03 23:15:54,435 storage ✅ 成功初始化 COSStorage
INFO 2025-02-03 23:15:54,436 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist


In [2]:
# 导入通过模块
from django.contrib.auth import get_user_model
from django.utils import timezone
from datetime import datetime, timedelta
import json 
from pprint import pprint


In [3]:
def print_json(data):
    """格式化打印JSON数据"""
    print(json.dumps(data, ensure_ascii=False, indent=2))

<font color="orange"> 1. 测试models.py </font>


In [3]:
# 导入模型
from apps.projects.models import Project, ProjectHistory

In [ ]:
# 创建测试用户（如果不存在）
User = get_user_model()

def get_or_create_test_user():
    try:
        user = User.objects.get(phone='13800138000')
    except User.DoesNotExist:
        user = User.objects.create_user(
            phone='13800138000',
            password='testpass123',
            username='测试用户'
        )
    return user

user = get_or_create_test_user()
print(user)


1.1 基础字段验证： 验证字段值是否正确保存？  验证默认值是否正确？<br>
1.2 自动生成字段验证： 验证字段值是否自动生成？ 验证生成规则是否正确？

In [17]:
    # 2. 创建项目
project = Project.objects.create(
    # id 在模型里未定义，默认采用django 自增主键 (待验证)
    # project_code='TEST-2024-001',  # 自动生成(待验证)
    project_name='测试项目',
    tenderee='测试招标单位',
    bidder='测试投标单位',
    project_type=Project.ProjectType.FSD,
    bid_deadline=timezone.now() + timedelta(days=30),
    # current_stage=Project.ProjectStage.DRAFT,  # 默认值待验证
    # is_urgent=True,  # 默认值未False 待验证
    creator=user
    # create_time=timezone.now(),  # 自动生成(待验证) 
    # last_update_time=timezone.now(),  # 自动生成(待验证)
    )



In [ ]:
print(f"\n创建项目成功: {project}")  # print(project.__str__())
print(f"项目id: {project.id}")
print(f"项目编号: {project.project_code}")
print(f"项目名称: {project.project_name}")
print(f"招标单位: {project.tenderee}")
print(f"投标单位: {project.bidder}")
print(f"项目类型: {project.get_project_type_display()}")
print(f"当前阶段: {project.get_current_stage_display()}")
print(f"是否紧急: {project.is_urgent}")
print(f"创建时间: {project.create_time}")
print(f"最后更新: {project.last_update_time}")
print(f"创建者: {project.creator}")


In [19]:
# 验证字段值是否正确保存
assert project.project_name == "测试项目"
assert project.project_type == Project.ProjectType.FSD
# 验证默认值是否正确
assert project.current_stage == Project.ProjectStage.DRAFT
assert project.is_urgent == False
# 验证UUID是否自动生成
assert project.id is not None
# 验证项目编号格式
assert project.project_code.startswith(f'BP-{timezone.now().year}')
# 验证时间字段是否自动设置
assert project.create_time is not None
assert project.last_update_time is not None

1.3 模型的方法测试： 测试状态自动变更， 验证历史记录。 

In [20]:
# 测试状态变更
old_stage = project.current_stage
project.current_stage = Project.ProjectStage.ANALYZING
project.save()

In [ ]:
print(f"状态更新成功: {old_stage} -> {project.current_stage}")
print(f"历史记录: {ProjectHistory.objects.filter(project=project)}")
print(f"操作时间: {ProjectHistory.objects.filter(project=project).first().operation_time}")
print(f"备注: {ProjectHistory.objects.filter(project=project).first().remarks}")

In [23]:
# 验证历史记录
history = ProjectHistory.objects.filter(project=project).first()
assert history.from_stage == old_stage
assert history.to_stage == Project.ProjectStage.ANALYZING

In [ ]:
# 清理测试数据
def cleanup_test_data():
    print("\n=== 清理测试数据 ===")
    Project.objects.filter(project_code='TEST-2024-001').delete()
    User.objects.filter(username='testuser').delete()
    print("测试数据已清理")

# 执行清理
cleanup_test_data()

<font color="orange"> 2. 测试serializers.py </font>

In [39]:
# 导入所需的模型和序列化器
from apps.projects.models import Project, ProjectHistory
from apps.projects.serializers import (
    UserBriefSerializer,
    ProjectHistorySerializer,
    ProjectListSerializer,
    ProjectDetailSerializer,
    ProjectCreateSerializer,
    ProjectUpdateSerializer,
    ProjectStageUpdateSerializer
)
from django.contrib.auth import get_user_model
from rest_framework.test import APIRequestFactory
from rest_framework.request import Request


2.1 序列化测试

In [31]:
#2.1 序列化测试

serializer = ProjectDetailSerializer(project)
data = serializer.data

In [ ]:
print(data['id'])
print(data['project_code'])
print(data['project_name'])
print(data['tenderee'])
print(data['bidder'])
print(data['project_type_display']) 
print(data['bid_deadline'])
print(data['current_stage_display'])
print(data['is_urgent'])
print(data['creator'])
print(data['create_time'])
print(data['last_update_time'])
#---------------------------------
print(data['stage_histories'])
print(data['stage_histories'][0]['id'])
print(data['stage_histories'][0]['project'])
print(data['stage_histories'][0]['from_stage_display'])
print(data['stage_histories'][0]['to_stage_display'])
print(data['stage_histories'][0]['operation_time'])
print(data['stage_histories'][0]['remarks'])


In [38]:
# 验证序列化后的数据
assert data['project_name'] == project.project_name
assert data['project_type_display'] == project.get_project_type_display()
assert 'stage_histories' in data


2.2 反序列化和验证测试

In [40]:
# 创建模拟请求的辅助函数
def create_request(url='/', user=None, presigned=False):
    factory = APIRequestFactory()
    if presigned:
        wsgi_request = factory.get(url + '?presigned=true')
    else:
        wsgi_request = factory.get(url)
    
    if user:
        wsgi_request.user = user
    else:
        # 如果没有提供用户，设置为 AnonymousUser  - 匿名用户 意味着没有登录，报错的原因
        from django.contrib.auth.models import AnonymousUser
        wsgi_request.user = AnonymousUser()    

    # 将 WSGIRequest 转换为 DRF Request，并确保用户信息被保留
    drf_request = Request(wsgi_request)
    drf_request.user = wsgi_request.user  # 显式设置用户
    
    # 将 WSGIRequest 转换为 DRF Request
    return drf_request

# 创建测试用户和基本请求
test_user = get_or_create_test_user()
request = create_request(user=test_user)

In [41]:
#2.2 反序列化测试
data = {
        'project_name': '测试项目',
        'tenderee': '测试招标单位',
        'bidder': '测试投标单位',
        'project_type': Project.ProjectType.FSD,
        'bid_deadline': timezone.now() + timedelta(days=30),
        'is_urgent': True
    }

serializer = ProjectCreateSerializer(
    data=data, 
    context={'request': request}
)

In [ ]:
print(serializer.is_valid())
project = serializer.save()
print(project)
print(project.creator)


2.3 更新序列化测试

In [45]:
update_data = {
    'project_name': '更新后的名称',
    'is_urgent': True
}

In [46]:
serializer = ProjectUpdateSerializer(
    project,
    data=update_data,
    partial=True
)
assert serializer.is_valid()
updated_project = serializer.save()
assert updated_project.project_name == update_data['project_name']

In [ ]:
print(updated_project)


2.4 状态更新序列化测试

In [ ]:
stage_data = {
    'current_stage': Project.ProjectStage.ANALYZING,
    'remarks': '开始分析文档'
}
serializer = ProjectStageUpdateSerializer(
    project,
    data=stage_data,
    partial=True
)
assert serializer.is_valid()
updated_project = serializer.save()
assert updated_project.current_stage == stage_data['current_stage']
print(updated_project)


In [ ]:
print(f"状态更新成功: {old_stage} -> {updated_project.current_stage}")
print(f"历史记录: {ProjectHistory.objects.filter(project=updated_project)}")
print(f"操作时间: {ProjectHistory.objects.filter(project=updated_project).first().operation_time}")
print(f"备注: {ProjectHistory.objects.filter(project=updated_project).first().remarks}")

In [ ]:
def cleanup_test_data():
    print("\n=== 清理测试数据 ===")
    Project.objects.filter(project_code__startswith='TEST-').delete()
    User.objects.filter(username='testuser').delete()
    print("测试数据已清理")


cleanup_test_data()

<font color="orange"> 3. 测试views.py </font>


In [4]:
# 导入所需的模型和工具
from apps.projects.models import Project, ProjectHistory
from apps.projects.views import ProjectViewSet
from rest_framework.test import APIClient # 模拟API请求
from rest_framework import status

In [5]:
# 创建测试用户（如果不存在）
User = get_user_model()

def get_or_create_test_user():
    try:
        user = User.objects.get(phone='13800138000')
    except User.DoesNotExist:
        user = User.objects.create_user(
            phone='13800138000',
            password='testpass123',
            username='测试用户'
        )
    return user

user = get_or_create_test_user()
print(user)

13800138000


In [6]:
# 创建API客户端, 创建模拟请求
client = APIClient()
client.force_authenticate(user=user)

print("\n=== 测试创建项目 ===")
project_data = {
        'project_name': '测试项目3',
        'tenderee': '测试招标单位',
        'bidder': '测试投标单位',
        #'project_type': Project.ProjectType.FSD,
        'bid_deadline': (timezone.now() + timedelta(days=30)).isoformat(),
        'is_urgent': True
}
    
response = client.post('/api/projects/', project_data, format='json')

if response.status_code == status.HTTP_201_CREATED:
    print("项目创建成功！")
    project_id = response.data['id']
else:
    print("创建失败:", response.data)
        


=== 测试创建项目 ===
项目创建成功！


In [7]:
print(response.data)

{'id': 12, 'project_code': 'BP-2025-OTHER-0010', 'project_name': '测试项目3', 'tenderee': '测试招标单位', 'bidder': '测试投标单位', 'project_type': 'OTHER', 'bid_deadline': '2025-03-05T15:16:03.284477Z', 'is_urgent': True}


In [8]:
 # 3. 测试获取项目列表
print("\n=== 测试获取项目列表 ===")
response = client.get('/api/projects/')
    
if response.status_code == status.HTTP_200_OK:
    print(f"获取到 {len(response.data)} 个项目")
    print_json(response.data)
else:
    print("获取列表失败:", response.data)



=== 测试获取项目列表 ===
获取到 10 个项目
[
  {
    "id": 12,
    "project_code": "BP-2025-OTHER-0010",
    "project_name": "测试项目3",
    "tenderee": "测试招标单位",
    "bidder": "测试投标单位",
    "project_type": "OTHER",
    "project_type_display": "其他",
    "bid_deadline": "2025-03-05T15:16:03.284477Z",
    "current_stage": "DRAFT",
    "current_stage_display": "草稿",
    "is_urgent": true,
    "creator": {
      "id": "38a2f8fd-abd3-4d66-97eb-19e1531c6577",
      "phone": "13800138000",
      "email": null,
      "role": "user"
    },
    "create_time": "2025-02-03T15:16:03.328673Z",
    "last_update_time": "2025-02-03T15:16:03.328673Z"
  },
  {
    "id": 11,
    "project_code": "BP-2025-OTHER-0009",
    "project_name": "测试项目3",
    "tenderee": "测试招标单位",
    "bidder": "测试投标单位",
    "project_type": "OTHER",
    "project_type_display": "其他",
    "bid_deadline": "2025-03-05T15:01:37.367371Z",
    "current_stage": "DRAFT",
    "current_stage_display": "草稿",
    "is_urgent": true,
    "creator": {
      "id": "

In [9]:
    # 4. 测试获取项目详情
project_id = 3
print("\n=== 测试获取项目详情 ===")
response = client.get(f'/api/projects/{project_id}/')

print(response.status_code)
print(status.HTTP_200_OK)
print(response.data)





=== 测试获取项目详情 ===
200
200
{'id': 3, 'project_code': 'TEST-2024-003', 'project_name': '更新后的项目名称', 'tenderee': '测试招标单位', 'bidder': '测试投标单位', 'project_type': 'FSD', 'project_type_display': '食材配送', 'bid_deadline': '2025-03-05T12:43:43.896735Z', 'current_stage': 'ANALYZING', 'current_stage_display': '分析中', 'is_urgent': False, 'creator': {'id': '38a2f8fd-abd3-4d66-97eb-19e1531c6577', 'phone': '13800138000', 'email': None, 'role': 'user'}, 'create_time': '2025-02-03T12:43:43.935984Z', 'last_update_time': '2025-02-03T15:10:48.320167Z', 'stage_histories': [{'id': 8, 'project': 3, 'from_stage': 'DRAFT', 'to_stage': 'ANALYZING', 'from_stage_display': '草稿', 'to_stage_display': '分析中', 'operation_time': '2025-02-03T15:10:48.318154Z', 'remarks': ''}, {'id': 7, 'project': 3, 'from_stage': 'DRAFT', 'to_stage': 'ANALYZING', 'from_stage_display': '草稿', 'to_stage_display': '分析中', 'operation_time': '2025-02-03T15:10:48.314591Z', 'remarks': '开始分析文档'}]}


In [10]:
# 5. 测试更新项目
print("\n=== 测试更新项目 ===")
update_data = {
    'project_name': '更新后的项目名称',
    'is_urgent': False
}
response = client.patch(f'/api/projects/{project_id}/', update_data, format='json')
    
if response.status_code == status.HTTP_200_OK:
    print("项目更新成功！")
    print_json(response.data)
else:
    print("更新失败:", response.data)


=== 测试更新项目 ===
项目更新成功！
{
  "project_name": "更新后的项目名称",
  "tenderee": "测试招标单位",
  "bidder": "测试投标单位",
  "project_type": "FSD",
  "bid_deadline": "2025-03-05T12:43:43.896735Z",
  "is_urgent": false
}


In [11]:
    # 6. 测试更新项目状态
print("\n=== 测试更新项目状态 ===")
stage_data = {
    'current_stage': Project.ProjectStage.ANALYZING,
    'remarks': '开始分析文档'
}
response = client.patch(f'/api/projects/{project_id}/update_stage/', stage_data, format='json')
print_json(response.data)



=== 测试更新项目状态 ===
{
  "id": 3,
  "project_code": "TEST-2024-003",
  "project_name": "更新后的项目名称",
  "tenderee": "测试招标单位",
  "bidder": "测试投标单位",
  "project_type": "FSD",
  "project_type_display": "食材配送",
  "bid_deadline": "2025-03-05T12:43:43.896735Z",
  "current_stage": "ANALYZING",
  "current_stage_display": "分析中",
  "is_urgent": false,
  "creator": {
    "id": "38a2f8fd-abd3-4d66-97eb-19e1531c6577",
    "phone": "13800138000",
    "email": null,
    "role": "user"
  },
  "create_time": "2025-02-03T12:43:43.935984Z",
  "last_update_time": "2025-02-03T15:16:13.903778Z",
  "stage_histories": [
    {
      "id": 8,
      "project": 3,
      "from_stage": "DRAFT",
      "to_stage": "ANALYZING",
      "from_stage_display": "草稿",
      "to_stage_display": "分析中",
      "operation_time": "2025-02-03T15:10:48.318154Z",
      "remarks": ""
    },
    {
      "id": 7,
      "project": 3,
      "from_stage": "DRAFT",
      "to_stage": "ANALYZING",
      "from_stage_display": "草稿",
      "to_stage_d

In [13]:
    # 7. 测试获取项目历史记录
print("\n=== 测试获取项目历史记录 ===")
response = client.get(f'/api/projects/{project_id}/histories/')
print_json(response.data)



=== 测试获取项目历史记录 ===
[
  {
    "id": 8,
    "project": 3,
    "from_stage": "DRAFT",
    "to_stage": "ANALYZING",
    "from_stage_display": "草稿",
    "to_stage_display": "分析中",
    "operation_time": "2025-02-03T15:10:48.318154Z",
    "remarks": ""
  },
  {
    "id": 7,
    "project": 3,
    "from_stage": "DRAFT",
    "to_stage": "ANALYZING",
    "from_stage_display": "草稿",
    "to_stage_display": "分析中",
    "operation_time": "2025-02-03T15:10:48.314591Z",
    "remarks": "开始分析文档"
  }
]


In [14]:
# 8. 测试删除项目
print("\n=== 测试删除项目 ===")
response = client.delete(f'/api/projects/{project_id}/')
    
if response.status_code == status.HTTP_204_NO_CONTENT:
    print("项目删除成功！")
else:
    print("删除失败:", response.data)

Bad Request: /api/projects/3/



=== 测试删除项目 ===
Bad Request: /api/projects/3/
删除失败: {'detail': '只能删除草稿或已取消的项目'}
